In [1]:
import jax
import jax.numpy as jnp
from jax import random, grad, jit, jacfwd, jacrev
from jax.scipy.linalg import inv, svd, eigh, det
from jax.lax import scan
from scipy.linalg import solve_discrete_are
from tqdm import tqdm
import matplotlib.pyplot as plt
from jax.tree_util import Partial
from functools import partial
from jax_vi import KL_gaussian, log_likelihood, KL_sum, plot_optimization_results
from jax_filters import apply_filtering_fixed_nonlinear, kalman_filter_process, filter_step_nonlinear
from jax_models import visualize_observations, Lorenz96, KuramotoSivashinsky, generate_true_states, generate_localization_matrix

key = random.PRNGKey(3)

# System dimensions
n = 40  # System dimension
p = 2  # Observation dimension
J0 = 0 # burn in period
N = 10 # Monte Carlo samples

num_steps = 1000  # Number of time steps

n = 256 # Dimensionality of the state space for KS model
observation_interval = 5  # Interval at which observations are made
dt = 0.25  # Time step for the KS model


# Model parameters
m0 = jnp.ones((n,))
C0 = jnp.eye(n) * 1.0   # Initial state covariance matrix (P)
q = random.normal(key, (n, n))/5
#Q = q@q.T + jnp.eye(n)*0.1    # Process noise covariance matrix (Sigma in Julia code) we use diagonal Q for nonlinear case
Q = jnp.eye(n)*0.1   #jnp.eye(n) * 5.0    # Process noise covariance matrix (Sigma in Julia code)

H = jnp.eye(n)          # Observation matrix
# H = jnp.eye(n)[::2] #partial observation

R = jnp.eye(H.shape[0])  # R now becomes 20x20 for partial H 20*40
inv_R = inv(R)


# State initialization
initial_state = random.normal(random.PRNGKey(0), (n,))  # Initial state
ks_model = KuramotoSivashinsky(dt=dt, s=n, l=22, M=16)

state_transition_function = ks_model.step
ks_step = Partial(state_transition_function)
key = random.PRNGKey(0)
jacobian_function = jacrev(ks_step, argnums=0)
jac_func = Partial(jacobian_function)


In [2]:
observations, true_states = generate_true_states(key, num_steps, n, initial_state, H, Q, R, ks_step, observation_interval)
y = observations

In [3]:
@partial(jit, static_argnums=(3,10))
def var_cost(K, m0, C0, n, H, Q, R, y, key, num_steps, J0):
    states, covariances = apply_filtering_fixed_nonlinear(m0, C0, y, K, n, ks_step, jac_func, H, Q, R)
    key, *subkeys = random.split(key, num=N+1)
    kl_sum = KL_sum(states, covariances, n, ks_step, Q, key, N)

    def inner_map(subkey):
        return log_likelihood(random.multivariate_normal(subkey, states, covariances), y, H, R, num_steps, J0) 
    cost = kl_sum - jnp.nanmean(jax.lax.map(inner_map, jnp.vstack(subkeys)))
    print(cost)
    return cost

@partial(jit, static_argnums=(3))
def var_cost_single_step(K, m0, C0, n, Q, H, R, y_curr, key, J, J0):
    (m_update, C_update), _  =  filter_step_nonlinear((m0,C0), y_curr, K, n, ks_step, jac_func, H, Q, R)
    
    log_likelihood_val = log_likelihood(m_update[jnp.newaxis, :], y_curr[jnp.newaxis, :], H, R, J=1, J0=J0)
    # Calculate the KL divergence between the predicted and updated state distributions
    m_pred = state_transition_function(m0)
    M = jac_func(m0)
    C_pred = M @ C0 @ M.T + Q
    kl_divergence = KL_gaussian(n, m_update, C_update, m_pred, C_pred)
    
    # Combine the KL divergence and the negative log-likelihood to form the cost
    cost = kl_divergence - log_likelihood_val
    return cost


In [ ]:
base_m, base_C, base_K = kalman_filter_process(ks_step, jac_func, m0, C0, observations, H, Q, R)
K_steady = jnp.mean(base_K[-10:, :, :], axis=0)


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from jax.numpy import linalg as jnpl
from tqdm.auto import tqdm

var_cost_grad = grad(var_cost, argnums=0)

# Initial guess for K and optimization parameters
K_opt = jnp.eye(n) * 0.4

alpha = 1e-6

live = False
prediction_errors = [] 
norms = []
true_div = []
last_200_errors = []

n_iters = 200
num_steps = n_iters

for i in tqdm(range(n_iters)):
    key, _ = random.split(key)
    # Update the gradient and Kalman gain
    grad_K = var_cost_grad(K_opt, m0, C0, n, H, Q, R, y, key, num_steps, J0)
    K_opt -= alpha * grad_K
    
    # Apply filtering with the newly optimized K to generate state predictions
    predicted_states, covariances = apply_filtering_fixed_nonlinear(m0, C0, y, K_opt, n, ks_step, jac_func, H, Q, R)
    
    prediction_error = np.mean(np.mean((predicted_states - true_states)**2, axis=1))
    prediction_errors.append(prediction_error)
    last_200_prediction_error = jnp.mean((predicted_states[-200:] - true_states[-200:])**2)
    last_200_errors.append(last_200_prediction_error)
    norms.append(jnpl.norm(K_opt - K_steady))
    total_kl_divergence = 0
    for t in range(num_steps):  
        kl_div_t = KL_gaussian(n, predicted_states[t], covariances[t],  base_m[t], base_C[t])
        total_kl_divergence += kl_div_t
    
    true_div.append(total_kl_divergence / num_steps)

In [ ]:
import os
from jax_vi import plot_optimization_results, plot_k_matrices

subfolder_name = 'ks_nonlinear_results'
os.makedirs(subfolder_name, exist_ok=True)  # Create directory if it doesn't exist
file_base_name = "ks_nonlinear_gain"
if H.shape[0] == 20:
    file_base_name += "_partial"
if live:
    file_base_name += "_live"
file_name = file_base_name + ".pdf"
file_path = os.path.join(subfolder_name, file_name)

plot_optimization_results(norms, prediction_errors, true_div, n_iters, ' ')
plot_k_matrices(K_steady, K_opt, ' ')




In [ ]:
print("Mean error for last 200 timesteps (at convergence)", jnp.mean(jnp.array(last_200_errors[:10])))

In [ ]:
import matplotlib.pyplot as plt
from jax.numpy import linalg as jnp
from tqdm.auto import tqdm


true_div = []
prediction_errors = [] 
norms = []
Ks = []
live = True

# Define the gradient of the cost function
var_cost_single_grad = grad(var_cost_single_step, argnums = 0)

# Initial guess for K and optimization parameters
alpha = 1e-5


if H.shape[0] == 40:
    K_opt = jnp.eye(n) * 0.4
else: #partial obs
    K_opt = jnp.zeros((40, 20))
    for i in range(0, K_opt.shape[1]):
        K_opt = K_opt.at[i*2, i].set(1)
    #K_opt = K_opt + random.normal(key, K_opt.shape) * 0.6

num_steps = 1000
for i in tqdm(range(num_steps)):
    key, _ = random.split(key)
    y_curr = observations[i] 
    # Update the gradient and Kalman gain
    for j in range(100):
        grad_K = var_cost_single_grad(K_opt, m0, C0, n, Q, H, R, y_curr, key, num_steps, J0)
        K_opt -= alpha * grad_K
    Ks.append(K_opt)
    norms.append(jnp.linalg.norm(K_opt - K_steady)) 
    (m_update, C_update), _ = filter_step_nonlinear((m0,C0), y_curr, K_opt, n, ks_step, jac_func, H, Q, R)
    prediction_error = jnp.square(m_update - true_states[i]).mean()  # Assuming true_states[i] is available
    prediction_errors.append(prediction_error)
    true_div.append(KL_gaussian(n, m_update, C_update, base_m[i], base_C[i]))
    # Prepare for the next step
    m0, C0 = m_update, C_update



In [ ]:
classic_kf_prediction_errors = jnp.mean(jnp.square(base_m - true_states), axis=1)

print("Average prediction error - Classic KF:", jnp.mean(classic_kf_prediction_errors[-200:]),
      "Gradient Descent KF:", jnp.mean(prediction_errors[-200:]))
